# 0. Imports and Setup

In [41]:
import pandas as pd
from sodapy import Socrata
from dotenv import load_dotenv
import os
from tqdm import tqdm

In [2]:
load_dotenv()
app_token = os.getenv("APP_TOKEN")

In [3]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(
    socrata_domain,
    app_token=app_token,
    timeout=1000
)

# 1. Get A List of Complaint Types To Track

In [4]:
query = """
SELECT 
    complaint_type,
    descriptor, 
    count(unique_key) AS count
GROUP BY 
    complaint_type, descriptor
ORDER BY 
    count DESC
LIMIT 
    3000
"""

results = client.get(
    socrata_dataset_identifier,
    query=query
)
client.close()

type_descriptor_df = pd.DataFrame(results)

print('shape of data: {}'.format(type_descriptor_df.shape))
type_descriptor_df.head(10)

shape of data: (2529, 3)


,complaint_type,descriptor,count
0,Noise - Residential,Loud Music/Party,2186708
1,HEAT/HOT WATER,ENTIRE BUILDING,1422046
2,Blocked Driveway,No Access,1148829
3,Request Large Bulky Item Collection,Request Large Bulky Item Collection,1073753
4,Noise - Street/Sidewalk,Loud Music/Party,915639
5,HEATING,HEAT,868960
6,Noise - Residential,Banging/Pounding,857874
7,Street Light Condition,Street Light Out,834121
8,HEAT/HOT WATER,APARTMENT ONLY,756633
9,Street Condition,Pothole,752024


In [6]:
type_descriptor_df.to_csv('types_descriptors.csv', index=False)

# 2. Get Timeseries Of Overall Complaints

In [8]:
query = """
SELECT 
    date_trunc_ymd(created_date) AS day, 
    count(unique_key) AS count
GROUP BY 
    day
ORDER BY 
    day DESC
LIMIT 
    10000
"""

results = client.get(
    socrata_dataset_identifier,
    query=query
)
client.close()

overall_df = pd.DataFrame(results)

print('shape of data: {}'.format(overall_df.shape))
overall_df.head(10)

shape of data: (5177, 2)


,day,count
0,2024-03-04T00:00:00.000,291
1,2024-03-03T00:00:00.000,8252
2,2024-03-02T00:00:00.000,6991
3,2024-03-01T00:00:00.000,8910
4,2024-02-29T00:00:00.000,8994
5,2024-02-28T00:00:00.000,8434
6,2024-02-27T00:00:00.000,8855
7,2024-02-26T00:00:00.000,8989
8,2024-02-25T00:00:00.000,7215
9,2024-02-24T00:00:00.000,7091


In [9]:
overall_df.to_csv('overall_ts.csv', index=False)

# 3. Get a buuuunch of data TKTK 

In [44]:
types_descriptors = pd.read_csv('types_descriptors.csv')

In [45]:
types_descriptors.head(5)

,complaint_type,descriptor,count
0,Noise - Residential,Loud Music/Party,2186708
1,HEAT/HOT WATER,ENTIRE BUILDING,1422046
2,Blocked Driveway,No Access,1148829
3,Request Large Bulky Item Collection,Request Large Bulky Item Collection,1073753
4,Noise - Street/Sidewalk,Loud Music/Party,915639


In [50]:
for index, row in tqdm(types_descriptors.iloc[404:].iterrows()):
    query = f"""
        SELECT 
            date_trunc_ymd(created_date) AS day, 
            count(unique_key) AS count
        WHERE
            complaint_type == "{row['complaint_type']}"
            AND descriptor == "{row['descriptor']}"
        GROUP BY 
            day
        ORDER BY 
            day DESC
        LIMIT 
            10000
        """
    results = client.get(socrata_dataset_identifier,query=query)
    client.close()
    df = pd.DataFrame(results)
    df['complaint_type'] = row['complaint_type']
    df['descriptor'] = row['descriptor']
    df.to_csv('data/'+str(index)+'.csv',index=False)

1720it [11:20,  2.53it/s]


HTTPError: 400 Client Error: Bad Request.
	Could not parse SoQL query "
        SELECT 
            date_trunc_ymd(created_date) AS day, 
            count(unique_key) AS count
        WHERE
            complaint_type == "..\..\..\..\..\..\..\..\..\..."
            AND descriptor == "Trauma Counseling"
        GROUP BY 
            day
        ORDER BY 
            day DESC
        LIMIT 
            10000
        ": Unexpected escape character:
            complaint_type == "..\..\..\..\..\..\..\..\..\..."
                                 ^

In [25]:
queries = [ f"""
SELECT 
    date_trunc_ymd(created_date) AS day, 
    count(unique_key) AS count
WHERE
    complaint_type == "{row['complaint_type']}"
    AND descriptor == "{row['descriptor']}"
GROUP BY 
    day
ORDER BY 
    day DESC
LIMIT 
    10000
""" for index, row in types_descriptors.iterrows()]

In [29]:
queries[0]

'\nSELECT \n    date_trunc_ymd(created_date) AS day, \n    count(unique_key) AS count\nWHERE\n    complaint_type == "Noise - Residential"\n    AND descriptor == "Loud Music/Party"\nGROUP BY \n    day\nORDER BY \n    day DESC\nLIMIT \n    10000\n'